In [1]:
!nvidia-smi

Tue Sep 26 11:03:19 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0     Off |                  N/A |
|  0%   34C    P8    14W / 200W |      0MiB /  8113MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:03:00.0     Off |                  N/A |
|  0%   34C    P8    15W / 200W |   7927MiB /  8112MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
import tensorflow as tf
import numpy as np
import sys
import os

sys.path.append(os.path.join("..", "..", ".."))
import ecg.dataset as ds
from ecg.batch import ModelEcgBatch

Using TensorFlow backend.


In [3]:
triplet_train_pipeline = (ds.Pipeline()
                          .init_variable("loss", init=list)
             .load(fmt="wfdb", components=["signal", "meta"])
             .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv", fmt="csv", components="target")
             .drop_labels(["~"])
             .drop_short_signals(6000)
             .get_triplets(100, 3000)
             .signal_transpose([0, 2, 1])
             .train_on_batch("triplet_learn")
             .run(batch_size=300, shuffle=True, drop_last=True, n_epochs=1, prefetch=0, lazy=True))

In [4]:
index = ds.FilesIndex(path="/notebooks/data/ECG/training2017/*.hea", no_ext=True, sort=True)
eds = ds.Dataset(index, batch_class=ModelEcgBatch)
eds.cv_split(0.8)
res = (eds.test >> triplet_train_pipeline).run()

In [5]:
res.get_variable('loss')

[0.60388583, 0.62872207, 0.55118024, 0.58133411, 0.55989772]

In [6]:
res.get_model_by_name('triplet_learn').hist

{'train_loss': [0.60388583, 0.62872207, 0.55118024, 0.58133411, 0.55989772],
 'train_metric': [0.0, 0.0, 0.0, 0.0, 0.0],
 'val_loss': [],
 'val_metric': []}

In [7]:
triplet_test_pipeline = (ds.Pipeline()
                          .init_variable("loss", init=list)
             .load(fmt="wfdb", components=["signal", "meta"])
             .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv", fmt="csv", components="target")
             .drop_labels(["~"])
             .drop_short_signals(6000)
             .get_triplets(100, 3000)
             .signal_transpose([0, 2, 1])
             .test_on_batch("triplet_learn")
             .run(batch_size=300, shuffle=True, drop_last=True, n_epochs=1, prefetch=0, lazy=True))

In [8]:
res_test = (eds.test >> triplet_test_pipeline).run()

In [12]:
triplet_predict_pipeline = (ds.Pipeline()
                          .init_variable("prediction", init=list)
             .load(fmt="wfdb", components=["signal", "meta"])
             .load(src="/notebooks/data/ECG/training2017/REFERENCE.csv", fmt="csv", components="target")
             .drop_labels(["~"])
             .drop_short_signals(6000)
             .get_triplets(100, 3000)
             .signal_transpose([0, 2, 1])
             .predict_on_batch("triplet_learn")
             .run(batch_size=300, shuffle=False, drop_last=False, n_epochs=1, prefetch=0, lazy=True))

In [13]:
res_predict = (eds.test >> triplet_predict_pipeline).run()

In [16]:
res_predict.get_variable('prediction')

[array([[ 0.9514128 ,  0.95962369],
        [ 0.97053468,  0.85033596],
        [ 0.83063549,  0.80462658],
        [ 0.95590711,  0.92671216],
        [ 0.94419944,  0.83236063],
        [ 0.89231551,  0.9354775 ],
        [ 0.80397332,  0.85722023],
        [ 0.96430051,  0.7852658 ],
        [ 0.93169785,  0.89728355],
        [ 0.82677507,  0.82168406],
        [ 0.90660954,  0.95642835],
        [ 0.96677995,  0.95553565],
        [ 0.94709104,  0.91338837],
        [ 0.90769929,  0.91570282],
        [ 0.98063046,  0.91294724],
        [ 0.89018977,  0.96211016],
        [ 0.94437313,  0.91366774],
        [ 0.95232576,  0.93463314],
        [ 0.91927952,  0.9017908 ],
        [ 0.95041305,  0.96352255],
        [ 0.97648573,  0.97669983],
        [ 0.9213382 ,  0.86868131],
        [ 0.96644092,  0.96099496],
        [ 0.94998848,  0.90034479],
        [ 0.97004223,  0.95751989],
        [ 0.96217191,  0.94610828],
        [ 0.960154  ,  0.93261456],
        [ 0.91498458,  0.973